import pandas, statistics, and scipy.stats, then read in the annotation files

In [37]:
import pandas as pd
from statistics import mean
import scipy.stats 

In [38]:
OPN1LW_df = pd.read_csv('OPN1LW_annotations.txt', sep='\t', header = 0, names = ['chr','cons','region','start','stop','junk1','junk2','junk3','id'])
OPN1MW_df = pd.read_csv('OPN1MW_annotations.txt', sep='\t', header = 0, names = ['chr','cons','region','start','stop','junk1','junk2','junk3','id'])
OPN1SW_df = pd.read_csv('OPN1SW_annotations.txt', sep='\t', header = 0, names = ['chr','cons','region','start','stop','junk1','junk2','junk3','id'])

chr7_var = pd.read_csv('chr7_variants.txt',sep='\t',header=0,names=['chr','start','stop','freqs','junk','junk2'])
chrX_var = pd.read_csv('chrX_variants.txt',sep='\t',header=0,names=['chr','start','stop','freqs','junk','junk2'])

take a look at the data contained in the annotation files

In [39]:
print(chr7_var.head())

   chr  start   stop                                              freqs junk  \
0    7  16486  16487  ERATE=0.0118;LDAF=0.8149;AN=2184;AC=1968;VT=SN...    .   
1    7  16510  16511  AA=.;AC=2155;AF=0.99;AFR_AF=0.95;AMR_AF=1.00;A...    .   
2    7  16654  16655  LDAF=0.3636;AN=2184;AC=418;VT=SNP;AA=.;ERATE=0...    .   
3    7  16718  16719  THETA=0.0164;RSQ=0.1774;AN=2184;VT=SNP;AA=.;SN...    .   
4    7  17711  17712  AA=.;AC=51;AF=0.02;AFR_AF=0.0020;AMR_AF=0.03;A...    .   

  junk2  
0     +  
1     +  
2     +  
3     +  
4     +  


retrieve protein-coding regions (CDS) only

In [40]:
OPN1LW_CDS = OPN1LW_df[OPN1LW_df['region']=='CDS'].copy()
OPN1MW_CDS = OPN1MW_df[OPN1MW_df['region']=='CDS'].copy()
OPN1SW_CDS = OPN1SW_df[OPN1SW_df['region']=='CDS'].copy()

In [41]:
print(OPN1LW_CDS)

     chr    cons region      start       stop junk1 junk2 junk3  \
2   chrX  HAVANA    CDS  154144284  154144395     .     +     0   
5   chrX  HAVANA    CDS  154150656  154150952     .     +     2   
7   chrX  HAVANA    CDS  154152940  154153108     .     +     2   
9   chrX  HAVANA    CDS  154154574  154154739     .     +     1   
11  chrX  HAVANA    CDS  154156294  154156533     .     +     0   
13  chrX  HAVANA    CDS  154158816  154158926     .     +     0   
24  chrX  HAVANA    CDS  154152942  154153108     .     +     0   
26  chrX  HAVANA    CDS  154154574  154154739     .     +     1   
28  chrX  HAVANA    CDS  154156294  154156344     .     +     0   
30  chrX  HAVANA    CDS  154158816  154158926     .     +     0   

                                                   id  
2   ID=CDS:ENST00000369951.9;Parent=ENST0000036995...  
5   ID=CDS:ENST00000369951.9;Parent=ENST0000036995...  
7   ID=CDS:ENST00000369951.9;Parent=ENST0000036995...  
9   ID=CDS:ENST00000369951.9;Parent=EN

retrieve the start and stop coordinates for each gene and load them to a list

In [42]:
OPN1LW_starts = OPN1LW_CDS['start'].tolist()
OPN1LW_stops = OPN1LW_CDS['stop'].tolist()
OPN1LW_coords = OPN1LW_starts + OPN1LW_stops
OPN1LW_coords.sort()
print(OPN1LW_coords)

[154144284, 154144395, 154150656, 154150952, 154152940, 154152942, 154153108, 154153108, 154154574, 154154574, 154154739, 154154739, 154156294, 154156294, 154156344, 154156533, 154158816, 154158816, 154158926, 154158926]


In [43]:
OPN1MW_starts = OPN1MW_CDS['start'].tolist()
OPN1MW_stops = OPN1MW_CDS['stop'].tolist()
OPN1MW_coords = OPN1MW_starts + OPN1MW_stops
OPN1MW_coords.sort()
print(OPN1MW_coords)

[154182678, 154182789, 154187770, 154188066, 154190054, 154190058, 154190222, 154190222, 154191688, 154191688, 154191853, 154191853, 154193408, 154193408, 154193458, 154193647, 154195930, 154195930, 154196040, 154196040, 154219816, 154219927, 154224908, 154225204, 154227192, 154227196, 154227360, 154227360, 154228828, 154228828, 154228993, 154228993, 154230548, 154230548, 154230598, 154230787, 154233070, 154233070, 154233180, 154233180, 154257620, 154257731, 154262712, 154263008, 154264996, 154265164, 154266632, 154266797, 154268352, 154268591, 154270874, 154270984]


In [44]:
OPN1SW_starts = OPN1SW_CDS['start'].tolist()
OPN1SW_stops = OPN1SW_CDS['stop'].tolist()
OPN1SW_coords = OPN1SW_starts + OPN1SW_stops
OPN1SW_coords.sort()
print(OPN1SW_coords)

[128772540, 128772659, 128773649, 128773888, 128774498, 128774663, 128774986, 128775154, 128775439, 128775781]


read in the DNA sequence files

In [45]:
with open("chr7.fa", "r") as file1:
    f1 = file1.read()
chr7 = f1.replace("\n","")

with open("chrX.fa", "r") as file2:
    f2 = file2.read()
chrX = f2.replace("\n","")


store the coordinates as start and stop values

In [46]:
SW_seq = ''
list_len = 0
while list_len < len(OPN1SW_coords):
	start = OPN1SW_coords[list_len]
	stop = OPN1SW_coords[list_len + 1]
	SW_seq += chr7[start:stop]
	list_len += 2
print(len(SW_seq))

1033


In [47]:
LW_seq = ''
list_len = 0
while list_len < len(OPN1LW_coords):
	start = OPN1LW_coords[list_len]
	stop = OPN1LW_coords[list_len + 1]
	LW_seq += chrX[start:stop]
	list_len += 2
print(len(LW_seq))

598


In [48]:
MW_seq = ''
list_len = 0
while list_len < len(OPN1MW_coords):
	start = OPN1MW_coords[list_len]
	stop = OPN1MW_coords[list_len + 1]
	MW_seq += chrX[start:stop]
	list_len += 2
print(len(MW_seq))

2289


transcribe to DNA to RNA

In [49]:
LW_rna = ""
for i in LW_seq:
    if i == "A":
        LW_rna += "U"
    elif i == "C":
        LW_rna += "G"
    elif i == "G":
        LW_rna += "C"
    elif i == "T":
        LW_rna += "A"

In [50]:
MW_rna = ""
for i in MW_seq:
    if i == "A":
        MW_rna += "U"
    elif i == "C":
        MW_rna += "G"
    elif i == "G":
        MW_rna += "C"
    elif i == "T":
        MW_rna += "A"

In [51]:
SW_rna = ""
for i in SW_seq:
    if i == "A":
        SW_rna += "U"
    elif i == "C":
        SW_rna += "G"
    elif i == "G":
        SW_rna += "C"
    elif i == "T":
        SW_rna += "A"

translate RNA to protein (dictionary was created by ChatGPT)

In [52]:
def translate(rna):
    table = {
        'UUU': 'F', 'UUC': 'F', 'UUA': 'L', 'UUG': 'L',
        'CUU': 'L', 'CUC': 'L', 'CUA': 'L', 'CUG': 'L',
        'AUU': 'I', 'AUC': 'I', 'AUA': 'I', 'AUG': 'M',
        'GUU': 'V', 'GUC': 'V', 'GUA': 'V', 'GUG': 'V',
        'UCU': 'S', 'UCC': 'S', 'UCA': 'S', 'UCG': 'S',
        'CCU': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
        'ACU': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
        'GCU': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
        'UAU': 'Y', 'UAC': 'Y', 'UAA': '*', 'UAG': '*',
        'CAU': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
        'AAU': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
        'GAU': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
        'UGU': 'C', 'UGC': 'C', 'UGA': '*', 'UGG': 'W',
        'CGU': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
        'AGU': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
        'GGU': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'
    }
    protein = []
    for i in range(0, len(rna), 3):
        codon = rna [i:i+3]
        amino = table.get(codon, 'Unknown')
        protein.append(amino)
    return protein

In [53]:
LW_proteins = translate(LW_rna)
MW_proteins = translate(MW_rna)
SW_proteins = translate(SW_rna)

extract the rows from the variant files that have the same start coordinate as the ones identified as pathogenic in dbSNP

In [54]:
LW_var_start = [154154734,154158844,154154602,154156440,154150812,154150897]

LW_var = chrX_var[chrX_var['start'].isin(LW_var_start)]

print(LW_var)

Empty DataFrame
Columns: [chr, start, stop, freqs, junk, junk2]
Index: []


In [55]:
MW_var_start = [154191716,154187939,154195934,154190173,154182566,154195934]

MW_var = chrX_var[chrX_var['start'].isin(MW_var_start)]

print(MW_var)

Empty DataFrame
Columns: [chr, start, stop, freqs, junk, junk2]
Index: []


In [56]:
SW_var_start = [128775556,128774545,128773786]

SW_var = chr7_var[chr7_var['start'].isin(SW_var_start)]

print(SW_var)

Empty DataFrame
Columns: [chr, start, stop, freqs, junk, junk2]
Index: []


pull in the overlapping variants after performing the bedtools intersection 

In [57]:
cds7var = pd.read_csv('cds7_overlapping_variants.txt', sep='\t',header=0,names=['chrom','start','stop','stuff','else','else2'])
cdsXvar = pd.read_csv('cdsX_overlapping_variants.txt', sep='\t',header=0,names=['chrom','start','stop','stuff','else','else2'])

now lets separate these based on the specific genes

In [58]:
cdsLWvar = cdsXvar[(cdsXvar['start'] >= 154144243) & (cdsXvar['stop'] <= 154159032) & (cdsXvar['stuff'].str.contains('VT=SNP'))]
cdsMWvar = cdsXvar[(cdsXvar['start'] >= 154182678) & (cdsXvar['stop'] <= 154196040)& (cdsXvar['stuff'].str.contains('VT=SNP'))]
cdsSWvar = cds7var[(cds7var['start'] >= 128772540) & (cds7var['stop'] <= 128775781)& (cds7var['stuff'].str.contains('VT=SNP'))]

parse for the population allelic frequencies (AF=)

In [59]:
goLW = cdsLWvar['stuff'].tolist()
goMW = cdsMWvar['stuff'].tolist()
goSW = cdsSWvar['stuff'].tolist()

In [60]:
print(goLW[0])

AA=G;AN=1659;AC=12;THETA=0.0005;VT=SNP;LDAF=0.0074;RSQ=0.9940;SNPSOURCE=LOWCOV;ERATE=0.0002;AVGPOST=0.9999;AF=0.01;AFR_AF=0.03


In [61]:
newLW = [i.split(';') for i in goLW]
newMW = [i.split(';') for i in goMW]
newSW = [i.split(';') for i in goSW]

print(newLW[0])

['AA=G', 'AN=1659', 'AC=12', 'THETA=0.0005', 'VT=SNP', 'LDAF=0.0074', 'RSQ=0.9940', 'SNPSOURCE=LOWCOV', 'ERATE=0.0002', 'AVGPOST=0.9999', 'AF=0.01', 'AFR_AF=0.03']


In [62]:
freqLW = []
for line in newLW:
    for item in line:
        if item.startswith('AF='):
            freqLW.append(item)

freqMW = []
for line in newMW:
    for item in line:
        if item.startswith('AF='):
            freqMW.append(item)

freqSW = []
for line in newSW:
    for item in line:
        if item.startswith('AF='):
            freqSW.append(item)

print(freqMW)

['AF=0.0012', 'AF=0.0006', 'AF=0.0006', 'AF=0.03', 'AF=0.0012', 'AF=0.0012', 'AF=0.0006', 'AF=0.01', 'AF=0.09', 'AF=0.01']


In [63]:
freqLWnum = []
for i in freqLW:
    numsplitLW = i.split('=')
    freqLWnum.append(numsplitLW[1])

freqMWnum = []
for i in freqMW:
    numsplitMW = i.split('=')
    freqMWnum.append(numsplitMW[1])

freqSWnum = []
for i in freqSW:
    numsplitSW = i.split('=')
    freqSWnum.append(numsplitSW[1])

freqLWnum = [float(i) for i in freqLWnum]
freqMWnum = [float(i) for i in freqMWnum]
freqSWnum = [float(i) for i in freqSWnum]

print(type(freqLWnum[0]))


<class 'float'>


average the frequencies for each gene

In [64]:
LWavg = mean(freqLWnum)
MWavg = mean(freqMWnum)
SWavg = mean(freqSWnum)

print(LWavg)
print(MWavg)
print(SWavg)

0.00454
0.014539999999999999
0.09644


perform anova test to see if there is significant difference between average allelic frequencies of each gene (assume alpha = 0.05)

In [65]:
test = scipy.stats.f_oneway(freqLWnum,freqMWnum,freqSWnum)
print(test)

F_onewayResult(statistic=1.1698314064712758, pvalue=0.32334282635205275)


since p-val>0.05, the 3 means are not significantly different